In [332]:
from pymongo import MongoClient
import pandas as pd
import requests
import os
import folium


In [333]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
companies = db.get_collection("Project III - companies")

In [17]:
companies.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [18]:
companies.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [19]:
company_offices = companies.distinct("offices")

for offices in company_offices:

    print(offices)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
design_companies_count = companies.count_documents({'category_code': 'design'})
print(f"Number of design companies: {design_companies_count}")

Number of design companies: 4


### Here I want to see what are some good markets to be located as a video game company:

In [20]:
# Condition for companies categorized as video games
condition_video_games = {"category_code": "games_video"}

# Projection to include desired fields
projection = {
    "_id": 0,
    "name": 1,
    "category_code": 1,
    "total_money_raised": 1,
    "number_of_employees": 1, 
    "country_code": {"$arrayElemAt": ["$offices.country_code", 0]},
    "city": {"$arrayElemAt": ["$offices.city", 0]},
    "address": {"$arrayElemAt": ["$offices.address1", 0]},
    "zip_code": {"$arrayElemAt": ["$offices.zip_code", 0]},
    "latitude": {"$arrayElemAt": ["$offices.latitude", 0]},  
    "longitude": {"$arrayElemAt": ["$offices.longitude", 0]}  
    
}

# Query the database with the condition for video games companies
video_games_companies = list(companies.find(condition_video_games, projection))

# Convert the list of dictionaries to a DataFrame
video_games_companies_df = pd.DataFrame(video_games_companies)

# Show the head of the DataFrame
video_games_companies_df

,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude
0,Flektor,games_video,NaN,$0,USA,Culver City,"8536 National Blvd, Suite A",90232,34.025958,-118.379768
1,Lala,games_video,NaN,$44.2M,USA,Palo Alto,209 Hamilton Ave,94301,37.451151,-122.154369
2,Joost,games_video,0.0,$45M,USA,New York,100 5th Ave Fl 6,10011-6903,40.746497,-74.009447
3,Babelgum,games_video,NaN,$13.2M,GBR,London,,,53.344104,-6.267494
4,Sparter,games_video,NaN,$0,USA,None,None,None,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...,...
1078,Symposium Productions,games_video,NaN,$0,NaN,NaN,NaN,NaN,NaN,NaN
1079,Rev Media,games_video,NaN,$0,USA,Dallas,,,NaN,NaN
1080,Fictionaut,games_video,2.0,$0,USA,Astoria,32-14 42nd St,11103,40.759795,-73.916127
1081,Tiny Speck,games_video,45.0,$17.2M,CAN,Vancouver,163 Hastings St W,BC V6B 1H5,49.282455,-123.109217


In [21]:
video_games_companies_df.to_csv('../data/video_games_companies.csv', index=False)

### Here I want to extract location from offices that fullfil developers requirements:

In [22]:
# Conditions to fullfill x% (designers and developers) of employees requirements
condition_design = {"category_code": {"$regex": "design", "$options": "i"}}
condition_funding = {"total_money_raised": {"$regex": "\\$[1-9]\\d{0,2}(M|B)", "$options": "i"}}
combined_conditions = {"$or": [condition_design, condition_funding]}

# Projection includes name, total money raised, country and city from the first office, and attempt to retrieve latitude and longitude
projection = {
    "_id": 0,
    "name": 1,
    "category_code": 1,
    "total_money_raised": 1,
    "country_code": {"$arrayElemAt": ["$offices.country_code", 0]},
    "city": {"$arrayElemAt": ["$offices.city", 0]},
    "address": {"$arrayElemAt": ["$offices.address1", 0]},
    "zip_code": {"$arrayElemAt": ["$offices.zip_code", 0]},
    "latitude": {"$arrayElemAt": ["$offices.latitude", 0]},  
    "longitude": {"$arrayElemAt": ["$offices.longitude", 0]}  
}

# Query the database
df = list(companies.find(combined_conditions, projection))

# Convert the list of dictionaries to a DataFrame
required_companies_df = pd.DataFrame(df)

# Show the DataFrame for companies requiered by designers + developers
required_companies_df.head()

,name,category_code,total_money_raised,country_code,city,address,zip_code,latitude,longitude
0,Digg,news,$45M,USA,San Francisco,135 Mississippi St,94107,37.764726,-122.394523
1,Joost,games_video,$45M,USA,New York,100 5th Ave Fl 6,10011-6903,40.746497,-74.009447
2,SpinVox,messaging,$106M,GBR,Buckinghamshire,"Wethered House, Pound Lane",SL7 2AF,NaN,NaN
3,AddThis,advertising,$73M,USA,Vienna,1595 Spring Hill Road,22182,38.926172,-77.245195
4,Mahalo,web,$21M,USA,Culver City,3525 Eastham Dr.,90232,34.017606,-118.487267


In [23]:
required_companies_df.to_csv('../data/required_companies.csv', index=False)

### Here I want to extract companies that can be use as our new office in the USA:

In [24]:
# Condition for companies categorized as video games
condition_country_code = {"offices.country_code": "USA"}

# Projection to include desired fields
projection = {
    "_id": 0,
    "name": 1,
    "category_code": 1,
    "total_money_raised": 1,
    "number_of_employees": 1, 
    "country_code": {"$arrayElemAt": ["$offices.country_code", 0]},
    "city": {"$arrayElemAt": ["$offices.city", 0]},
    "address": {"$arrayElemAt": ["$offices.address1", 0]},
    "zip_code": {"$arrayElemAt": ["$offices.zip_code", 0]},
    "latitude": {"$arrayElemAt": ["$offices.latitude", 0]},  
    "longitude": {"$arrayElemAt": ["$offices.longitude", 0]}  
    
}

# Query the database with the condition for video games companies
usa_companies = list(companies.find(condition_country_code, projection))

# Convert the list of dictionaries to a DataFrame
usa_companies_df = pd.DataFrame(usa_companies)

# Show the head of the DataFrame
usa_companies_df

,name,category_code,number_of_employees,total_money_raised,country_code,city,address,zip_code,latitude,longitude
0,Wetpaint,web,47.0,$39.8M,USA,Seattle,710 - 2nd Avenue,98104,47.603122,-122.333253
1,AdventNet,enterprise,600.0,$0,USA,Pleasanton,4900 Hopyard Rd.,94588,37.692934,-121.904945
2,Zoho,software,1600.0,$0,USA,Pleasanton,4900 Hopyard Rd,94588,37.692934,-121.904945
3,Digg,news,60.0,$45M,USA,San Francisco,135 Mississippi St,94107,37.764726,-122.394523
4,Facebook,social,5299.0,$2.43B,USA,Menlo Park,1601 Willow Road,94025,37.416050,-122.151801
...,...,...,...,...,...,...,...,...,...,...
9261,Contemporary Computer Services,software,NaN,$0,USA,Bohemia,200 Knickerbocker Avenue,11716,40.775055,-73.088140
9262,QSGI,software,164.0,$0,USA,Palm Beach,400 Royal Palm Way,33480,26.705331,-80.041395
9263,AfterLogic,software,NaN,$0,USA,Livingston,P.O. Box 2096,07039,40.793024,-74.323554
9264,EnteGreat Solutions,software,NaN,$0,USA,Birmingham,,,33.518885,-86.816068


In [25]:
usa_companies_df.to_csv('../data/usa_companies.csv', index=False)

## GEOCODE API

I want to get the latitud and longitude that are missing from my preselected and required companies.

In [44]:
import os
from dotenv import load_dotenv
load_dotenv() # load_env

True

In [45]:
token = os.getenv("token")

In [29]:
MAX_ATTEMPTS = 3

def gets_coordinates_from_string (address, attemps):
    if attemps > MAX_ATTEMPTS:
        return None
    
    url_geocode = f"https://geocode.xyz/{address}?json=1"
    res = requests.get(url_geocode)
    
    try:
        response = res.json()
        
        # The API can fail in two ways, it either didn't find the address or the call was throttled
        if 'latt' in response and 'Throttled' not in response['latt']:
            return response
    except ex:
        print(f"Couldn't get geo data for {address}, attempts: {attempts}, error: {str(ex)}")
        
    # If we get here we haven't found geo data yet
    time.sleep(10)
    return gets_coordinates_from_string(address,attemps+1)


In [ ]:
row = {'address': '201 Third Street, 2nd Floor', 'city': 'San Francisco'}
full_address = f"{row['address']}, {row['city']}"
test1 = gets_coordinates_from_string(full_address, 0)
test1

In [ ]:
# First, filter out rows where latitude and longitude are missing
#companies_to_geocode = all_companies_cleaned_df[all_companies_cleaned_df['latitude'].isnull() | all_companies_cleaned_df['longitude'].isnull()]

# Then loop through these companies and geocode them
for index, row in all_companies_cleaned_df.iterrows():    
    row_lat = all_companies_cleaned_df.at[index, 'latitude']
    row_lng = all_companies_cleaned_df.at[index, 'longitude']
    address = all_companies_cleaned_df.at[index, 'address']
    
    lat_or_lng_empty = pd.isna(row_lat) and pd.isna(row_lng)
    has_address = type(address) == str
    
    # We only add information to rows that don't have lat and lng, that also have an address for search
    if lat_or_lng_empty and has_address:
        city = all_companies_cleaned_df.at[index, 'city']
        
        # We concat address and city to increase reliability
        full_address = f"{address}, {city}"
        print(f"Getting the geo data for {full_address}")
        
        response = gets_coordinates_from_string(full_address, 0)
        
        if response is None:
            print(f"Couldn't get geo data for address {full_address}")
            continue
        
        api_lat = float(response['latt'])
        api_lng = float(response['longt'])

        # Update the DataFrame with the new latitude and longitude
        all_companies_cleaned_df.at[index, 'latitude'] = api_lat
        all_companies_cleaned_df.at[index, 'longitude'] = api_lng

        # API rate limit
        time.sleep(5)

# Save the updated DataFrame
all_companies_cleaned_df.to_csv('../data/all_companies_cleaned_df.csv', index=False)

## FOURSQUARE API

In [264]:
def requests_for_foursquare(categories, lat, lon, radius_dict, limit=10):
    token = os.getenv("token")
    all_venues = []

    for query in categories:
        # Get the radius for the current category, default to 500 if not specified
        radius = radius_dict.get(query, 500)
        url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&sort=DISTANCE&limit={limit}"

        headers = {
            "accept": "application/json",
            "Authorization": token
        }

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # This will raise an exception for HTTP errors
            f4_answer = response.json()
        except requests.exceptions.HTTPError as errh:
            print("Http Error:", errh)
            continue
        except requests.exceptions.ConnectionError as errc:
            print("Error Connecting:", errc)
            continue
        except requests.exceptions.Timeout as errt:
            print("Timeout Error:", errt)
            continue
        except requests.exceptions.RequestException as err:
            print("Oops: Something Else", err)
            continue

        answer = f4_answer["results"]

        for element in answer:
            venue_name = element["name"]
            venue_address = element.get("location", {}).get("address", "")
            venue_distance = element["distance"]
            venue_city = element.get('location', {}).get('locality', '')
            venue_category = query
            venue_lat = element["geocodes"]["main"]["latitude"]
            venue_lon = element["geocodes"]["main"]["longitude"]

            small_dict = {
                "name": venue_name,
                "address": venue_address,
                "city": venue_city,
                "category": venue_category,
                "distance": venue_distance,
                "lat": venue_lat,
                "lon": venue_lon
            }

            all_venues.append(small_dict)

    df = pd.DataFrame(all_venues)
    return df


## 1. FIRST FINALIST : HI5

In [325]:
san_francisco_df = pd.read_csv('../data/san_francisco_companies_cleaned_df.csv')

In [326]:
hi5_info = san_francisco_df[san_francisco_df['name'].str.lower() == 'hi5']
hi5_lat = hi5_info['latitude'].iloc[0]
hi5_lon = hi5_info['longitude'].iloc[0]

In [327]:
categories = [
    'karaoke', 
    'bars', 
    'clubs', 
    'starbucks', 
    'vegan restaurants', 
    'elementary school',
    'basketball stadium',
    'veterinary',
    'pet grooming',
]

radius_dict = {
    'karaoke': 500,
    'bars': 500,
    'clubs': 500,
    'starbucks': 500,
    'vegan restaurants': 500,
    'elementary school': 700,
    'basketball stadium': 10000, 
    'veterinary':2000,
    'pet grooming': 2000,
}

# Latitude and Longitude for hi5
hi5_latitude = hi5_lat  
hi5_longitude = hi5_lon 

# Call the function
hi5_venues_df = requests_for_foursquare(categories, hi5_latitude, hi5_longitude,radius_dict)


In [329]:
hi5_venues_df

,name,address,city,category,distance,lat,lon
0,Bar Fluxus,18 Harlan Pl,San Francisco,karaoke,427,37.790297,-122.405140
1,Sool Bar and Lounge,323 Grant Ave,San Francisco,karaoke,470,37.789993,-122.405695
2,Uno Dos Tacos,595 Market St,San Francisco,bars,42,37.789274,-122.400993
3,Tequila Mockingbird,86 2nd St,San Francisco,bars,63,37.788120,-122.400280
4,Sammy's on 2nd,84 2nd St,San Francisco,bars,64,37.788120,-122.400280
...,...,...,...,...,...,...,...
73,Duckworth Peters Lebowitz Olivier,100 Bush St,San Francisco,pet grooming,386,37.791610,-122.399912
74,Petrino Mediterranean Cuisine,322 Kearny St,San Francisco,pet grooming,402,37.791110,-122.403897
75,P.L.A.Y. Pet Lifestyle And You,246 2nd St,San Francisco,pet grooming,408,37.785801,-122.397275
76,Peter Panos Bespoke Tailor,109 Geary St,San Francisco,pet grooming,430,37.787608,-122.405268


In [330]:
# Rows to be removed by index
rows_to_remove = [10, 12, 32, 44, 45, 46, 47, 48, 51, 53, 65,69,70,71,72,73,74,75,76,77]

# Drop the rows
hi5_venues_df = hi5_venues_df.drop(rows_to_remove, errors='ignore')

In [334]:
# New places to add to the dataframe
new_places = [
    {"name": "Doggie Day Spa", "address": "840 Harrison St", "city": "San Francisco", 
     "category": "pet grooming", "distance": 1287, "lat": 37.780172, "lon": -122.4034464},
    {"name": "City Dogs", "address": "177 Brannan St", "city": "San Francisco", 
     "category": "pet grooming", "distance": 1448, "lat": 37.7839701, "lon": -122.3917156}
]

# Convert the new places to a DataFrame
new_places_df = pd.DataFrame(new_places)

# Append the new places to the existing dataframe
hi5_venues_df_updated = hi5_venues_df.append(new_places_df, ignore_index=True)
hi5_venues_df_updated


AttributeError: 'DataFrame' object has no attribute 'append'

In [295]:
hi5_venues_df.to_csv('../data/hi5_venues_df.csv', index=False)

## 2. SECOND FINALIST : Zynga

In [296]:
zynga_info = san_francisco_df[san_francisco_df['name'].str.lower() == 'zynga']
z_lat = zynga_info['latitude'].iloc[0]
z_lon = zynga_info['longitude'].iloc[0]

In [297]:
categories = [
    'karaoke', 
    'bars', 
    'clubs', 
    'starbucks', 
    'vegan restaurants', 
    'elementary school',
    'basketball stadium',
    'veterinary',
    'pet grooming',
]

radius_dict = {
    'karaoke': 500,
    'bars': 500,
    'clubs': 500,
    'starbucks': 500,
    'vegan restaurants': 500,
    'elementary school': 700,
    'basketball stadium': 10000, 
    'veterinary':500,
    'pet grooming': 500,
}

# Latitude and Longitude for hi5
zynga_latitude = z_lat  
zynga_longitude = z_lon 

# Call the function
zynga_venues_df = requests_for_foursquare(categories, zynga_latitude, zynga_longitude, radius_dict)

In [298]:
zynga_venues_df

,name,address,city,category,distance,lat,lon
0,CCA Wattis Institute for Contemporary Arts,360 Kansas St,San Francisco,bars,33,37.765297,-122.403953
1,Live Sushi Bar,2001 17th St,San Francisco,bars,71,37.764500,-122.403691
2,PARAGON Real Estate Group - Potrero Hill,350 Rhode Island St,San Francisco,bars,101,37.765346,-122.402881
3,Starbucks,350 Rhode Island St,San Francisco,bars,105,37.765796,-122.403409
4,Next Level Clucker San Francisco,450 Rhode Island St,San Francisco,bars,133,37.764592,-122.402908
5,Steep Brew,450 Rhode Island St,San Francisco,bars,139,37.764500,-122.402831
6,Allegro Coffee Company,450 Rhode Island St,San Francisco,bars,146,37.764435,-122.402827
7,Whole Foods,450 Rhode Island St,San Francisco,bars,167,37.764381,-122.402680
8,Dos Pinas,251 Rhode Island St,San Francisco,bars,188,37.766745,-122.402533
9,Two Sons Sandwiches,2249 17th St,San Francisco,bars,198,37.764407,-122.406194


In [299]:
zynga_venues_df.to_csv('../data/zynga_venues_df.csv', index=False)

## 3. THIRD FINALIST : iWin

In [300]:
iwin_info = san_francisco_df[san_francisco_df['name'].str.lower() == 'iwin']
i_lat = iwin_info['latitude'].iloc[0]
i_lon = iwin_info['longitude'].iloc[0]

In [301]:
categories = [
    'karaoke', 
    'bars', 
    'clubs', 
    'starbucks', 
    'vegan restaurants', 
    'elementary school',
    'basketball stadium',
    'veterinary',
    'pet grooming',
]

radius_dict = {
    'karaoke': 500,
    'bars': 500,
    'clubs': 500,
    'starbucks': 500,
    'vegan restaurants': 500,
    'elementary school': 700,
    'basketball stadium': 10000, 
    'veterinary':500,
    'pet grooming': 500,
}


# Latitude and Longitude for hi5
iwin_latitude = i_lat  
iwin_longitude = i_lon 

# Call the function
iwin_venues_df = requests_for_foursquare(categories, zynga_latitude, zynga_longitude, radius_dict)

In [302]:
iwin_venues_df

,name,address,city,category,distance,lat,lon
0,CCA Wattis Institute for Contemporary Arts,360 Kansas St,San Francisco,bars,33,37.765297,-122.403953
1,Live Sushi Bar,2001 17th St,San Francisco,bars,71,37.764500,-122.403691
2,PARAGON Real Estate Group - Potrero Hill,350 Rhode Island St,San Francisco,bars,101,37.765346,-122.402881
3,Starbucks,350 Rhode Island St,San Francisco,bars,105,37.765796,-122.403409
4,Next Level Clucker San Francisco,450 Rhode Island St,San Francisco,bars,133,37.764592,-122.402908
5,Steep Brew,450 Rhode Island St,San Francisco,bars,139,37.764500,-122.402831
6,Allegro Coffee Company,450 Rhode Island St,San Francisco,bars,146,37.764435,-122.402827
7,Whole Foods,450 Rhode Island St,San Francisco,bars,167,37.764381,-122.402680
8,Dos Pinas,251 Rhode Island St,San Francisco,bars,188,37.766745,-122.402533
9,Two Sons Sandwiches,2249 17th St,San Francisco,bars,198,37.764407,-122.406194


In [303]:
iwin_venues_df.to_csv('../data/iwin_venues_df.csv', index=False)